<a href="https://colab.research.google.com/github/IlyaZutler/Bus_lanes/blob/main/DM%20_%20LinesGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString, Point
import folium
from geopy.distance import geodesic

In [93]:
# 'trips.xlsx' is uploaded to Colab environment
df_trips = pd.read_excel('trips.xlsx', header=0)  # header=0 means the first row is the header
df_trips.sample(2)

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition
1756,77403 (М947ОК67) Renault Logan,17.02.2025 22:58:41,"55,743981°, 37,814446°",20,+
2936,77403 (М947ОК67) Renault Logan,18.02.2025 15:01:54,"55,765946°, 37,832476°",5,+


In [94]:
# Split the 'Coordinates' column into 'Longitude' and 'Latitude'
df_trips['Сoordinates'] = df_trips['Сoordinates'].str.replace('°', '').str.replace(',', '.')
df_trips[['Longitude', 'Latitude']] = df_trips['Сoordinates'].str.split('. ', expand=True)
# df_trips.drop(['Сoordinates'], axis=1, inplace=True)

# Convert 'Day_time' to datetime objects
df_trips['Day_time'] = pd.to_datetime(df_trips['Day_time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')

df_trips['Ignition'] = df_trips['Ignition'].str.replace('-', '0').str.replace('+', '1')

df_trips = df_trips.sort_values(['Car_ID', 'Day_time'])  # Сортируем по времени

# Преобразование поездок в геометрию
df_trips['geometry'] = df_trips.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
df_trips = gpd.GeoDataFrame(df_trips, geometry='geometry', crs="EPSG:4326")

df_trips.sample(2)

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Longitude,Latitude,geometry
557,77403 (М947ОК67) Renault Logan,2025-02-17 09:37:06,55.765626. 37.832595,41,1,55.765626,37.832595,POINT (55.76563 37.8326)
351,77403 (М947ОК67) Renault Logan,2025-02-17 07:47:39,55.778238. 37.82606,11,1,55.778238,37.82606,POINT (55.77824 37.82606)


In [95]:
# Загрузка данных о Выделенных полосах
gdf_bus_lanes = gpd.read_file("bus-lanes.geojson")
gdf_bus_lanes = gdf_bus_lanes.set_crs("EPSG:4326")

In [96]:
# Добавляем buffer(0.0005) к выделенным полосам - примерно 30 метров
gdf_bus_lanes = gdf_bus_lanes.to_crs(epsg=3857)  # Web Mercator (метры)
gdf_bus_lanes["geometry"] = gdf_bus_lanes["geometry"].buffer(30)  # 30 метров (0.0003 градусов ~ 30 м)
gdf_bus_lanes = gdf_bus_lanes.to_crs(epsg=4326)  # Возвращаем в исходную систему

# Определяем пересечения с учетом buffer
df_trips["on_bus_lane"] = df_trips.intersects(gdf_bus_lanes["geometry"].union_all())

In [100]:
# Анализ скорости на участках
df_trips["next_time"] = df_trips.groupby('Car_ID')['Day_time'].shift(-1)
df_trips["next_lon"] = df_trips.groupby('Car_ID')['Longitude'].shift(-1)
df_trips["next_lat"] = df_trips.groupby('Car_ID')['Latitude'].shift(-1)

# Вычисляем расстояние между последовательными точками в км
# Создаем два массива для координат
coords1 = np.column_stack((df_trips['Latitude'], df_trips['Longitude']))
coords2 = np.column_stack((df_trips['next_lat'], df_trips['next_lon']))

# Функция для вычисления расстояний
def calculate_distances(coords1, coords2):
    return np.array([geodesic((lat1, lon1), (lat2, lon2)).meters / 1000
                     if not pd.isna(lat2) else np.nan
                     for (lat1, lon1), (lat2, lon2) in zip(coords1, coords2)])

# Применяем функцию для всей таблицы
df_trips['distance'] = calculate_distances(coords1, coords2)



# Вычисляем время между последовательными точками, часов, Заменяем NaN на None
time_diff = (df_trips["next_time"] - df_trips['Day_time']).dt.total_seconds() / 3600
df_trips["time"] = time_diff.where(df_trips["next_time"].notna(), None)



# Проверяем, чтобы значения в столбцах "distance" и "time" были валидными, и вычисляем скорость
df_trips["speed"] = np.where(
    (pd.notnull(df_trips["distance"])) & (df_trips["time"].notnull()) & (df_trips["time"] != 0),
    df_trips["distance"] / df_trips["time"],
    None
)

In [99]:
df_trips.iloc[236:239]

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Longitude,Latitude,geometry,on_bus_lane,next_time,next_lon,next_lat,distance,time,speed
236,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:27,55.767596. 37.83425,15,1,55.767596,37.83425,POINT (55.7676 37.83425),False,2025-02-17 06:53:31,55.76751,37.834183,0.010612,0.001111,9.550852
237,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:31,55.76751. 37.834183,7,1,55.76751,37.834183,POINT (55.76751 37.83418),False,2025-02-17 06:53:32,55.767491,37.834136,0.005478,0.000278,19.721727
238,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:32,55.767491. 37.834136,14,1,55.767491,37.834136,POINT (55.76749 37.83414),False,2025-02-17 06:53:33,55.767483,37.834058,0.008686,0.000278,31.269875


In [87]:
# Оценка скорости на выделенных полосах
avg_speed_bus_lane = 50

# Рассчитываем предсказанное время для каждой строки
df_trips['predicted_time'] = np.where(
    df_trips["on_bus_lane"],
    np.minimum(df_trips['time'], df_trips['distance'] / avg_speed_bus_lane),
    df_trips['time']
)

In [88]:
df_trips.iloc[236:239]

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Longitude,Latitude,geometry,on_bus_lane,next_time,next_lon,next_lat,distance,time,speed,predicted_time
236,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:27,55.767596. 37.83425,15,1,55.767596,37.83425,POINT (55.7676 37.83425),False,2025-02-17 06:53:31,55.76751,37.834183,0.010612,0.001111,9.550852,0.001111
237,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:31,55.76751. 37.834183,7,1,55.76751,37.834183,POINT (55.76751 37.83418),False,2025-02-17 06:53:32,55.767491,37.834136,0.005478,0.000278,19.721727,0.000278
238,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:32,55.767491. 37.834136,14,1,55.767491,37.834136,POINT (55.76749 37.83414),False,2025-02-17 06:53:33,55.767483,37.834058,0.008686,0.000278,31.269875,0.000278


In [91]:
# время в движении, часов
total_time = df_trips.loc[df_trips['speed'] > 0, 'time'].sum()

# Выигрыш во времени
df_trips["time_saved"] = df_trips["time"] - df_trips["predicted_time"]
total_time_saved = df_trips["time_saved"].sum()

print(f"Общая экономия времени: {total_time :.2f} часов")
print(f"% экономии времени: {total_time_saved / total_time  * 100  :.2f} %")



Общая экономия времени: 6.98 часов
% экономии времени: 0.00 %
